In [1]:
import xarray as xr
from cdo import Cdo
import sys
import os
import re
import glob
from functools import reduce
import pandas as pd
import cftime

# Import useful xarray recipes

In [2]:
sys.path.append('/home/abhi/Documents/mygit/postBC_diagnostic_bak/src/scripts/')
from recipes import *

# Enter the data directory

In [3]:
DATA_DIR = '/media/abhi/My_Passport/NEX-GDDP-NASA'

# Walk through the directory

In [21]:
dir_struct = {}

In [45]:
def get_directory_structure(rootdir):
    """
    Creates a nested dictionary that represents the folder structure of rootdir
    """
    dir_dict = {}
    rootdir = rootdir.rstrip(os.sep)
    start = rootdir.rfind(os.sep) + 1
    for path, dirs, files in os.walk(rootdir):
        folders = path[start:].split(os.sep)
        subdir = dict.fromkeys(files)
        parent = reduce(dict.get, folders[:-1], dir_dict)
        parent[folders[-1]] = subdir
    return dir_dict, subdir, folders
    
    
    

In [46]:
d = get_directory_structure(DATA_DIR)[2]

# Turn off cftime index

In [4]:
xr.set_options(enable_cftimeindex=False)

/home/abhi/miniconda3/lib/python3.6/site-packages/xarray/core/options.py:50: FutureWarning: The enable_cftimeindex option is now a no-op and will be removed in a future version of xarray.
  FutureWarning)


# Test ens mean with two models

In [26]:
m1 = xr.open_mfdataset(f'{DATA_DIR}/ACCESS1-0/pr/*hist*.nc')

In [4]:
m_correct = xr.open_mfdataset(f'{DATA_DIR}/BCC-CSM1-1/pr/*hist*.nc')

In [24]:
ti = m2.indexes['time']

In [31]:
ti.to_datetimeindex()

/home/abhi/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  """Entry point for launching an IPython kernel.


DatetimeIndex(['1950-01-01 12:00:00', '1950-01-02 12:00:00',
               '1950-01-03 12:00:00', '1950-01-04 12:00:00',
               '1950-01-05 12:00:00', '1950-01-06 12:00:00',
               '1950-01-07 12:00:00', '1950-01-08 12:00:00',
               '1950-01-09 12:00:00', '1950-01-10 12:00:00',
               ...
               '2005-12-22 12:00:00', '2005-12-23 12:00:00',
               '2005-12-24 12:00:00', '2005-12-25 12:00:00',
               '2005-12-26 12:00:00', '2005-12-27 12:00:00',
               '2005-12-28 12:00:00', '2005-12-29 12:00:00',
               '2005-12-30 12:00:00', '2005-12-31 12:00:00'],
              dtype='datetime64[ns]', length=20440, freq=None)

In [34]:
ti_wrong = m1.indexes['time']

AttributeError: 'DatetimeIndex' object has no attribute 'to_datetimeindex'

In [35]:
try:
    ti_wrong = ti_wrong.to_datetimeindex()
except AttributeError:
    pass

In [58]:
m2.indexes['time'].to_datetimeindex()

/home/abhi/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  """Entry point for launching an IPython kernel.


DatetimeIndex(['1950-01-01 12:00:00', '1950-01-02 12:00:00',
               '1950-01-03 12:00:00', '1950-01-04 12:00:00',
               '1950-01-05 12:00:00', '1950-01-06 12:00:00',
               '1950-01-07 12:00:00', '1950-01-08 12:00:00',
               '1950-01-09 12:00:00', '1950-01-10 12:00:00',
               ...
               '2005-12-22 12:00:00', '2005-12-23 12:00:00',
               '2005-12-24 12:00:00', '2005-12-25 12:00:00',
               '2005-12-26 12:00:00', '2005-12-27 12:00:00',
               '2005-12-28 12:00:00', '2005-12-29 12:00:00',
               '2005-12-30 12:00:00', '2005-12-31 12:00:00'],
              dtype='datetime64[ns]', length=20440, freq=None)

In [59]:
m2.time.values = m2.indexes['time'].to_datetimeindex()

/home/abhi/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  """Entry point for launching an IPython kernel.


In [66]:
m1.reindex_like(m2).time

<xarray.DataArray 'time' (time: 20440)>
array([cftime.DatetimeNoLeap(1950, 1, 1, 12, 0, 0, 0, 4, 1),
       cftime.DatetimeNoLeap(1950, 1, 2, 12, 0, 0, 0, 5, 2),
       cftime.DatetimeNoLeap(1950, 1, 3, 12, 0, 0, 0, 6, 3), ...,
       cftime.DatetimeNoLeap(2005, 12, 29, 12, 0, 0, 0, 1, 363),
       cftime.DatetimeNoLeap(2005, 12, 30, 12, 0, 0, 0, 2, 364),
       cftime.DatetimeNoLeap(2005, 12, 31, 12, 0, 0, 0, 3, 365)], dtype=object)
Coordinates:
  * time     (time) object 1950-01-01 12:00:00 ... 2005-12-31 12:00:00
Attributes:
    standard_name:  time
    long_name:      time
    axis:           T
    bounds:         time_bnds

In [39]:

for m in [m1, m2]:
    try:
        m.indexes['time'] = m.indexes['time'].to_datetimeindex()
    except AttributeError:
        pass

/home/abhi/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  after removing the cwd from sys.path.


TypeError: 'Indexes' object does not support item assignment

In [13]:
modelIndex = pd.Index(['BNU-ESM', 'BCC-CSM-1'], name='model')

In [11]:
m1.time

<xarray.DataArray 'time' (time: 20440)>
array([cftime.DatetimeNoLeap(1950, 1, 1, 12, 0, 0, 0, 4, 1),
       cftime.DatetimeNoLeap(1950, 1, 2, 12, 0, 0, 0, 5, 2),
       cftime.DatetimeNoLeap(1950, 1, 3, 12, 0, 0, 0, 6, 3), ...,
       cftime.DatetimeNoLeap(2005, 12, 29, 12, 0, 0, 0, 1, 363),
       cftime.DatetimeNoLeap(2005, 12, 30, 12, 0, 0, 0, 2, 364),
       cftime.DatetimeNoLeap(2005, 12, 31, 12, 0, 0, 0, 3, 365)], dtype=object)
Coordinates:
  * time     (time) object 1950-01-01 12:00:00 ... 2005-12-31 12:00:00
Attributes:
    standard_name:  time
    long_name:      time
    axis:           T
    bounds:         time_bnds

In [12]:
m2.time

<xarray.DataArray 'time' (time: 20440)>
array([cftime.DatetimeNoLeap(1950, 1, 1, 12, 0, 0, 0, 4, 1),
       cftime.DatetimeNoLeap(1950, 1, 2, 12, 0, 0, 0, 5, 2),
       cftime.DatetimeNoLeap(1950, 1, 3, 12, 0, 0, 0, 6, 3), ...,
       cftime.DatetimeNoLeap(2005, 12, 29, 12, 0, 0, 0, 1, 363),
       cftime.DatetimeNoLeap(2005, 12, 30, 12, 0, 0, 0, 2, 364),
       cftime.DatetimeNoLeap(2005, 12, 31, 12, 0, 0, 0, 3, 365)], dtype=object)
Coordinates:
  * time     (time) object 1950-01-01 12:00:00 ... 2005-12-31 12:00:00
Attributes:
    standard_name:  time
    long_name:      time
    axis:           T
    bounds:         time_bnds

In [16]:
%time
ens = xr.concat([m1, m2], dim=modelIndex)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.58 µs


In [18]:
ens_mean = ens.mean(dim='model')

In [20]:
%%time
ens_mean = ens_mean.compute()

CPU times: user 32.3 s, sys: 10.2 s, total: 42.4 s
Wall time: 35.2 s


In [97]:
dt  = m2.indexes['time'].to_datetimeindex()

/home/abhi/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  """Entry point for launching an IPython kernel.


In [120]:
m1.indexes['time'].unique()

DatetimeIndex(['1950-01-01 12:00:00', '1950-01-02 12:00:00',
               '1950-01-03 12:00:00', '1950-01-04 12:00:00',
               '1950-01-05 12:00:00', '1950-01-06 12:00:00',
               '1950-01-07 12:00:00', '1950-01-08 12:00:00',
               '1950-01-09 12:00:00', '1950-01-10 12:00:00',
               ...
               '2099-12-22 12:00:00', '2099-12-23 12:00:00',
               '2099-12-24 12:00:00', '2099-12-25 12:00:00',
               '2099-12-26 12:00:00', '2099-12-27 12:00:00',
               '2099-12-28 12:00:00', '2099-12-29 12:00:00',
               '2099-12-30 12:00:00', '2099-12-31 12:00:00'],
              dtype='datetime64[ns]', name='time', length=54787, freq=None)

In [122]:
m2.indexes['time'].to_datetimeindex().unique()

/home/abhi/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  """Entry point for launching an IPython kernel.


DatetimeIndex(['1950-01-01 12:00:00', '1950-01-02 12:00:00',
               '1950-01-03 12:00:00', '1950-01-04 12:00:00',
               '1950-01-05 12:00:00', '1950-01-06 12:00:00',
               '1950-01-07 12:00:00', '1950-01-08 12:00:00',
               '1950-01-09 12:00:00', '1950-01-10 12:00:00',
               ...
               '2099-12-22 12:00:00', '2099-12-23 12:00:00',
               '2099-12-24 12:00:00', '2099-12-25 12:00:00',
               '2099-12-26 12:00:00', '2099-12-27 12:00:00',
               '2099-12-28 12:00:00', '2099-12-29 12:00:00',
               '2099-12-30 12:00:00', '2099-12-31 12:00:00'],
              dtype='datetime64[ns]', length=54750, freq=None)

In [92]:
dt_series = pd.date_range('1950-01-01', '2099-12-31')

In [111]:
dt_series[np.isin(dt_series.year, dt.year)]

DatetimeIndex(['1950-01-01', '1950-01-02', '1950-01-03', '1950-01-04',
               '1950-01-05', '1950-01-06', '1950-01-07', '1950-01-08',
               '1950-01-09', '1950-01-10',
               ...
               '2099-12-22', '2099-12-23', '2099-12-24', '2099-12-25',
               '2099-12-26', '2099-12-27', '2099-12-28', '2099-12-29',
               '2099-12-30', '2099-12-31'],
              dtype='datetime64[ns]', length=54787, freq='D')

In [115]:
dt_series

DatetimeIndex(['1950-01-01', '1950-01-02', '1950-01-03', '1950-01-04',
               '1950-01-05', '1950-01-06', '1950-01-07', '1950-01-08',
               '1950-01-09', '1950-01-10',
               ...
               '2099-12-22', '2099-12-23', '2099-12-24', '2099-12-25',
               '2099-12-26', '2099-12-27', '2099-12-28', '2099-12-29',
               '2099-12-30', '2099-12-31'],
              dtype='datetime64[ns]', length=54787, freq='D')

# Load all the models lazily

In [5]:
sys.path.append('../scripts')

In [6]:
from helper_funcs import models, get_model_files

['ACCESS1-0',
 'BCC-CSM1-1',
 'BNU-ESM',
 'CanESM2',
 'CCSM4',
 'CESM1-BGC',
 'CNRM-CM5',
 'CSIRO-Mk3-6-0',
 'GFDL-CM3',
 'GFDL-ESM2G',
 'GFDL-ESM2M',
 'INMCM4',
 'IPSL-CM5A-LR',
 'IPSL-CM5A-MR',
 'MIROC-ESM',
 'MIROC-ESM-CHEM',
 'MIROC5',
 'MPI-ESM-LR',
 'MPI-ESM-MR',
 'MRI-CGCM3',
 'NorESM1-M']

In [7]:
from collections import OrderedDict

In [8]:
dsets = OrderedDict({})

In [9]:
%%time

variable = 'pr'
scen_name = 'historical'
for model in models:
    dsets[model] = xr.open_mfdataset(f'{DATA_DIR}/{model}/{variable}/*{scen_name}*.nc')

CPU times: user 23 s, sys: 552 ms, total: 23.6 s
Wall time: 1min 4s


In [97]:
ti.values


array(['1950-01-01T12:00:00.000000000', '1950-01-02T12:00:00.000000000',
       '1950-01-03T12:00:00.000000000', ...,
       '2005-12-29T12:00:00.000000000', '2005-12-30T12:00:00.000000000',
       '2005-12-31T12:00:00.000000000'], dtype='datetime64[ns]')

In [ ]:
m2.time.values = m2.indexes['time'].to_datetimeindex()

In [10]:
%%time

for model, dset in dsets.items():
    try:
        dset.time.values = dset.indexes['time'].to_datetimeindex()
    except AttributeError:
        pass
    finally:
        _, index = np.unique(dset['time'], return_index=True)
        dset = dset.isel(time=index)
        dset = dset.reindex_like(m_correct)
        dsets[model] = dset

/home/abhi/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  after removing the cwd from sys.path.


CPU times: user 4.18 s, sys: 8 ms, total: 4.19 s
Wall time: 4.39 s


In [11]:
ti_counts  = {model:ds.time.size for model, ds in dsets.items()}

In [12]:
ti_counts

{'ACCESS1-0': 20440,
 'BCC-CSM1-1': 20440,
 'BNU-ESM': 20440,
 'CanESM2': 20440,
 'CCSM4': 20440,
 'CESM1-BGC': 20440,
 'CNRM-CM5': 20440,
 'CSIRO-Mk3-6-0': 20440,
 'GFDL-CM3': 20440,
 'GFDL-ESM2G': 20440,
 'GFDL-ESM2M': 20440,
 'INMCM4': 20440,
 'IPSL-CM5A-LR': 20440,
 'IPSL-CM5A-MR': 20440,
 'MIROC-ESM': 20440,
 'MIROC-ESM-CHEM': 20440,
 'MIROC5': 20440,
 'MPI-ESM-LR': 20440,
 'MPI-ESM-MR': 20440,
 'MRI-CGCM3': 20440,
 'NorESM1-M': 20440}

In [13]:
modelIndex = pd.Index(models, name='model')

In [15]:
%%time
ens_mean = xr.concat(dsets.values(), dim=modelIndex).mean(dim='model').compute()

MemoryError: 